In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

In [19]:
# 1. Load text embedding model (CPU-friendly)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Connect to Qdrant (assumes localhost; change if needed)
client = QdrantClient("http://localhost:6333")  # or use `host`, `port` args

# 3. Create the collection if it doesn’t exist
COLLECTION_NAME = "APARAVI"
VECTOR_DIM = 384  # all-MiniLM-L6-v2 output size

if not client.collection_exists('APARAVI'):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=VECTOR_DIM, distance=Distance.COSINE)
    )

# 4. Sample DataFrame
df = pd.read_csv("./master_links.csv")
# 5. Generate embeddings
embeddings = model.encode([str(text) for text in df["text"].tolist()]).tolist()

# 6. Convert to Qdrant Points
points = [
    PointStruct(
        id=int(i),
        vector=vec,
        payload={
            "text": df.loc[i, "text"],
            "link": df.loc[i, "link"]
        }
    )
    for i, vec in enumerate(embeddings)
]

# 7. Upsert to Qdrant
operation_info = client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=points
)

print(operation_info)


operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [20]:
def get_context_embedding(company_context: str):
    """Precompute the embedding for the company context"""
    return model.encode(company_context, convert_to_tensor=False)

COMPANY_CONTEXT="""We mainly focus on two areas:

PAD (Project Advisory and Disputes): Typically in the context of construction projects, we mostly offer dispute support, serving as an advisor to quantify damages or claims. We dive deep into construction schedules to perform “delay analysis”, seeing how one event impacted the next and so forth. We then do “quantitative analysis”, to tie figures into the analysis once we have established causality. On the advisory side, we are always looking for opportunities to assist large infrastructure projects on their management practices, but we are not specialized engineers or architects (although some of us do have that kind of background); so we are not going to perform engineering tasks. Just management advisory, risk advisory, and dispute advisory.
Transactions: We make due diligence reports for companies looking to sell or acquire businesses. This entails reviewing trial balances, management accounts, and reconciling and performing quality of earnings (QoE) adjustments to offer a view of normalized EBITDA. We are not financial auditors or accountants.
Strategy: specifically in AI and healthcare, we offer strategic support and presentations to managers to assist them with managerial decisions and keep them updated on latest trends. This is the most lenient type of work available to us, and we will do strategy projects in different industries – we recently worked with an agriculture firm to advise them on AI impacts on their business. So the window is very broad."""
search_result = client.query_points(
    collection_name="APARAVI",
    query= get_context_embedding(COMPANY_CONTEXT),
    with_payload=True,
    limit=3 
).points

for res in search_result:
    print(res.model_dump())

{'id': 23, 'version': 0, 'score': 0.48161975, 'payload': {'text': 'PROCESS, TRANSACTION AND FINANCIAL ADVISORY SERVICES', 'link': 'https://urldefense.com/v3/__https://www.merx.com/private/supplier/interception/open-solicitation/3448885067?utm_medium=email&utm_source=system_2025-05-23_AM&utm_campaign=savedsearch_notifications&pid=17220162&utm_content=abstract_443523236325&language=EN__;!!CKcPr9h30w!exGRQ6F3nX0YHwj_p1Q5_Hq_7UsLhrKIM4uNZGC-smhLze9SZhK16WBGZHtQcf4ZwR9LB4VL1AO3HUsc$'}, 'vector': None, 'shard_key': None, 'order_value': None}
{'id': 167, 'version': 0, 'score': 0.4189157, 'payload': {'text': 'Consultant Services - Mechanical Engineers', 'link': 'https://urldefense.com/v3/__https://email.biddingo.com/c/eJxMjkFqwzAQAF8jH8VqtbLkgw6BYGhp6aEvWGntRGBbQXFi-vtCLu11mIGRiD1L7qZoPPkAYIG6awwGJ2fFDuJSImQ35Fmytx5mnAO4rkSD2NNAfiA9QMDw1wNY_c5P_uSy6LLNVRGkIlK2S9W5rt0Sr_t-uyt7UjgqHI_j0P8FheO0clk-6qVsyo5zq6uy51TktExt_36ke24lTU1f91Vhn4p8tcubKHsmAxS8cwr7fZP2YsaAJQq2a_HW6q3eedGZNxZWBJzzo3H-eY09I_4G